In [41]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForQuestionAnswering

# model_ckpt = "albert-xxlarge-v2" 
# model_ckpt = "deepset/minilm-uncased-squad2"
model_ckpt = "gaussalgo/xlm-roberta-large_extractive-QA_en-cs"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

In [69]:
questions = ["Co se vám líbilo?", "Co se vám nelíbilo?", "Jaké byly postavy?", "Jaký byl děj?", "Jaké máte pocity z filmu?", "Jste spokojeni nebo zklamáni?", "Co si myslíte o Cameronových filmech?", "Co jste měli dnes k obědu?"]
context = """Zklamání, čekal jsem víc. Nikdy jsem nebyl žádnej modrej fanatik, ale před třinácti lety jsem se nechal tou mánií strhnout, na jedničku šel na 3D a dal jí pět hvězd. Teď jsem si jedničku zopakoval a zůstal jsem na pěti - je to pořád jenom blbá pohádka, ale z různých důvodů pohádka hodná pěti hvězd. A byl jsem zvědavej, jakým směrem se bude ubírat dvojka - třináctileté "čekání", obrovské plány, obrovské přípravy, zvěsti o opětovné revolučnosti atd. atd... A dostal jsem prachobyčejnou dvojku jako přes kopírák. Jaká asi bude zápletka? Aha, lidi se vrátili, takže zase hodní Na´vi versus zlí lidi, jako v jedničce, OK. Kdo asi bude hlavní záporák? Aha, Quaritch sice v jedničce umřel, ale máme ho tu zase, OK. Tak si alespoň ušetříme hodinové objevování světa Pandory jako v jedničce. Aha, ze stromů se přesuneme do vody a uděláme to samé jako v jedničce, OK. (Navštívíme ve trojce sněžné kmeny, ve čtyřce jeskynní kmeny, v pětce ohňové kmeny...? Budeme v každém díle hodinu objevovat nepoznané kraje Pandory?) Kdyby mne zajímaly problémy dospívajících a jejich výchovy, vyhledal bych ve filmotéce příslušné tituly a nevydával se na tříhodinovou pouť vesmírem... 50%"""

In [70]:
print(tokenizer.decode(inputs["input_ids"][0]))

<s> Co jste měli dnes k obědu?</s></s> Dlouhé čekání se vyplatilo. Cameron nám po 13-ti letech naservíroval skvělou akční podívanou s úchvatným vizuálem. Stačí to ale? První Avatar většině lidem vyrazil dech, a proto byla laťka dost vysoko. Teď jsme vlastně dostali to samé, jen v jiném prostředí, delší a s rodinnou atmosférou. Zase asi těžko se dalo čekat něco převratného, příběh vám dech nevyrazí. Zato vizuál ano. Opravdu se to vyplatí vidět ve 3D. Děj se posouvá o zhruba 15 let dopředu, kdy Jake a Neytyri mají rodinu a šťastně žijí na Pandoře. Lidé ze Země se ale vrací a chtějí odplatu. Stojí proti nim stejná jednotka, jen ne jako lidi, ale mají těla Na'viů. Jake se proto s rodinnou uchýlí k vodnímu kmeni, kde se snaží schovat. To ale nejde běžně a nemůže to skončit jinak než velkým konfliktem. Trochu mi vadila délka, protože se prostředek filmu podle mě trochu vlekl. Jinak je to ale skvělá podívaná pro celou rodinu. Opět se tady trochu hraje na ekologickou notu a ochranu zvířat a př

In [71]:
print(f"Context: {context}")

for question in questions:
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_idx = torch.argmax(start_logits)  
    end_idx = torch.argmax(end_logits) + 1  
    answer_span = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_span)
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    answers = pipe(question=question, context=context, topk=3, handle_impossible_answer=True)
    
    print(f"Question: {question}")
    for idx, answer in enumerate(answers):
        print(f"\tTop {idx+1} answer: {answer['answer']}")

Context: Zklamání, čekal jsem víc. Nikdy jsem nebyl žádnej modrej fanatik, ale před třinácti lety jsem se nechal tou mánií strhnout, na jedničku šel na 3D a dal jí pět hvězd. Teď jsem si jedničku zopakoval a zůstal jsem na pěti - je to pořád jenom blbá pohádka, ale z různých důvodů pohádka hodná pěti hvězd. A byl jsem zvědavej, jakým směrem se bude ubírat dvojka - třináctileté "čekání", obrovské plány, obrovské přípravy, zvěsti o opětovné revolučnosti atd. atd... A dostal jsem prachobyčejnou dvojku jako přes kopírák. Jaká asi bude zápletka? Aha, lidi se vrátili, takže zase hodní Na´vi versus zlí lidi, jako v jedničce, OK. Kdo asi bude hlavní záporák? Aha, Quaritch sice v jedničce umřel, ale máme ho tu zase, OK. Tak si alespoň ušetříme hodinové objevování světa Pandory jako v jedničce. Aha, ze stromů se přesuneme do vody a uděláme to samé jako v jedničce, OK. (Navštívíme ve trojce sněžné kmeny, ve čtyřce jeskynní kmeny, v pětce ohňové kmeny...? Budeme v každém díle hodinu objevovat nepo